# Use AutoRunner to execute Auto3Dseg with minimal inputs

## Prepare libraries and download datasets

In [ ]:
import os

from monai.apps.auto3dseg import AutoRunner
from monai.bundle.config_parser import ConfigParser
from monai.apps import download_and_extract

In [ ]:
root = "./"
msd_task = "Task05_Prostate"
resource = "https://msd-for-monai.s3-us-west-2.amazonaws.com/" + msd_task + ".tar"
compressed_file = os.path.join(root, msd_task + ".tar")
if os.path.exists(root):
    download_and_extract(resource, compressed_file, root)

## Prepare the input YAML configuration for the datasets

In [ ]:
data_src_cfg = {
    "name": "Task05_Prostate",
    "task": "segmentation",
    "modality": "MRI",
    "datalist": "../tasks/msd/Task05_Prostate/msd_task05_prostate_folds.json",  # data traning, validation, testing splits in a JSON file
    "dataroot": "./Task05_Prostate",  # location of the datasets
    "multigpu": True,
    "class_names": ["val_acc_pz", "val_acc_tz"]
}
input = './input.yaml'
ConfigParser.export_config_file(data_src_cfg, input)

## Execute Auto3Dseg pipeline (data analysis, algo generation, training and ensemble) in AutoRunner

In [ ]:
work_dir = "./work_dir"
runner = AutoRunner(work_dir=work_dir, input=input)
runner.run()

### Use cached result to save time

In the Auto3Dseg, intermediate results are saved by default. The user can choose if they want to use the cached results or restart from scratch.

To skip some steps and use cache result, user can use the following arguments:

In [ ]:
# skip analyze and use cache. If there is no cache, AutoRunner will throw error
runner = AutoRunner(work_dir=work_dir, input=input, analyze=False, not_use_cache=False)  

By default AutoRunner will use cached results in the working directory. If the users want to start from scratch, they can set `not_use_cache` to True

In [ ]:
runner = AutoRunner(work_dir=work_dir, input=input, not_use_cache=True)  

### Output Result Formats

AutoRunner will perform inference on the testing data specified by the `datalist` in the data source config input. The inference result will be written `ensemble_output` folder under the working directory in the form of `nii.gz`. The user can choose the format by adding keyword arguments to the AutoRunner. A list of argument can be found in https://docs.monai.io/en/stable/transforms.html#saveimage.

In [ ]:
runner = AutoRunner(work_dir=work_dir, input=input, output_dir='./output_dir')

### Setting Auto3Dseg parameters

In [ ]:
# set number of folds for cross validation
runner = AutoRunner(work_dir=work_dir, input=input)
runner.set_num_fold(num_fold = 2)

In [ ]:
# set training parameters
runner = AutoRunner(work_dir=work_dir, input=input)
train_param = {
    "num_iterations": 8,
}
runner.set_training_params(params=train_param) 

# In the segresnet takes "num_epochs" instead of "num_iterations" for the training parameters.


In [ ]:
# set ensemble method
runner = AutoRunner(work_dir=work_dir, input=input)
runner.set_ensemble_method(ensemble_method_name="AlgoEnsembleBestByFold")

In [ ]:
# set model ensemble method
pred_params = {
    'files_slices': slice(0,2),  # only infer the first two files in the testing data
    'mode': "vote",              # use majority vote instead of mean to ensemble the predictions
    'sigmoid': True,             # when to use sigmoid to binarize the prediction and output the label
}
runner = AutoRunner(work_dir=work_dir, input=input)
runner.set_prediction_params(params=pred_params)

## Train model with HPO (Grid-search)

Auto3Dseg supports hyper parameter optimization (HPO) via NNI backends. The user can define a grid search space and automate the HPO. 

Note: to run the HPO, the `nni` package is required. You can use `pip install nni` to obtain the package.

In [ ]:
runner = AutoRunner(work_dir=work_dir, input=input, hpo=True)
search_space = {"learning_rate": {"_type": "choice", "_value": [0.0001, 0.001, 0.01, 0.1]}}
runner.set_nni_search_space(search_space)
runner.run()